<a href="https://www.kaggle.com/code/aryan401/indianfoodcnn?scriptVersionId=142203477" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split
from time import time
from statistics import mean

In [ ]:
tranform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/indian-food-16/Train', transform=tranform)
test_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/indian-food-16/Test', transform=tranform)
valid_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/indian-food-16/Validate', transform=tranform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print('Train DataSet len:',len(train_dataset))
print('Valid DataSet len:',len(valid_dataset))
print('Test DataSet len:',len(test_dataset))
print('Total DataSet Classes:',len(test_dataset.classes))

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.ConvSet_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.FC_Layers = nn.Sequential(
            nn.Linear(401408, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 16),
        )
    def forward(self, x):
        out = self.ConvSet_1(x)
        out = self.ConvSet_2(out)
        out = out.reshape(out.shape[0], -1)
        out = self.FC_Layers(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN()
model = model.to(device=device)
print(model)
print('Device: ',device)

In [ ]:
learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [ ]:
num_epochs = 50 # 1+ 5 + 20 + 20
print(f"Train DataLoader Size: {len(train_dataloader)}")
print(f"Valid DataLoader Size: {len(valid_dataloader)}")
for epoch in range(num_epochs):
    global_time = time()
    local_time_chg = []
    loss_ep = 0
    for images, labels in train_dataloader:
        local_time = time()
        images = images.to(device=device)
        labels = labels.to(device=device) 
        optimizer.zero_grad()
        scores = model(images)
        loss = criterion(scores,labels)
        loss.backward()
        optimizer.step()
        loss_ep += loss.item()
        local_time_chg.append(time()-local_time)
        if len(local_time_chg) % 10 == 0:
            print(f"Step Verified: {len(local_time_chg)}")
        
    with torch.no_grad():
        correct = 0
        total = 0
        print("Starting Validation Test...")
        for images, labels in valid_dataloader:
            images = images.to(device=device)
            labels = labels.to(device=device) 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy of the Model on the {len(valid_dataset)} Valid Images: {100*(correct /total)} %')

    print (f'Epoch [{epoch+1}/{num_epochs}] | Loss: {round(loss_ep/len(train_dataloader), 5)} | Step Time: {mean(local_time_chg)} | Epoch Time: {time() - global_time}')

In [ ]:
torch.save(model.state_dict(), "/kaggle/working/IndianFoodCNN.pt")
model = CNN()

In [ ]:
model.load_state_dict(torch.load("/kaggle/working/IndianFoodCNN.pt"))
model = model.to(device=device)
model.eval()

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device=device)
        labels = labels.to(device=device) 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the Model on the {} Test Images: {}%'.format(len(test_dataset), 100*correct /total))